<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Praisonai/praisonai_ai_agents_worlfllow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Job Market Analysis using Automated Web Search using PraisonAI
##Goal
This notebook implements an automated web search and validation system to analyze AI job trends in 2024. It uses a dual-agent workflow to:

1. Collect current data about AI job trends through DuckDuckGo searches
2. Automatically validate the search results for completeness and quality

The system leverages the PraisonAI framework to orchestrate the search and validation tasks, using Groq's LLM API for processing and analysis. The workflow is designed to automatically retry data collection if the validation criteria aren't met, ensuring robust and complete results.

In [7]:
%pip install -qU praisonai duckduckgo-search

In [13]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_BASE_URL'] = 'https://api.groq.com/openai/v1'

os.environ['OPENAI_MODEL_NAME'] = 'llama-3.3-70b-versatile'

In [15]:
from praisonaiagents import Agent, Task, PraisonAIAgents
from typing import List, Dict
from duckduckgo_search import DDGS

# 1. Tool
def internet_search_tool(query: str) -> List[Dict]:
    """
    Perform a search using DuckDuckGo.

    Args:
        query (str): The search query.

    Returns:
        list: A list of search result titles, URLs, and snippets.
    """
    try:
        results = []
        ddgs = DDGS()
        for result in ddgs.text(keywords=query, max_results=10):
            results.append({
                "title": result.get("title", ""),
                "url": result.get("href", ""),
                "snippet": result.get("body", "")
            })
        return results

    except Exception as e:
        print(f"Error during DuckDuckGo search: {e}")
        return []

# 2. Agent
data_agent = Agent(
    name="DataCollector",
    role="Search Specialist",
    goal="Perform internet searches to collect relevant information.",
    backstory="Expert in finding and organising internet data.",
    tools=[internet_search_tool],
    self_reflect=False
)

# 3. Tasks
collect_task = Task(
    description="Perform an internet search using the query: 'AI job trends in 2024'. Return results as a list of title, URL, and snippet.",
    expected_output="List of search results with titles, URLs, and snippets.",
    agent=data_agent,
    name="collect_data",
    is_start=True,
    next_tasks=["validate_data"]
)

validate_task = Task(
    description="""Validate the collected data. Check if:
    1. At least 5 results are returned.
    2. Each result contains a title and a URL.
    Return validation_result as 'valid' or 'invalid' only no other text.""",
    expected_output="Validation result indicating if data is valid or invalid.",
    agent=data_agent,
    name="validate_data",
    task_type="decision",
    condition={
        "valid": [exit],  # End the workflow on valid data
        "invalid": ["collect_data"]  # Retry data collection on invalid data
    },
)

# 4. Workflow
agents = PraisonAIAgents(
    agents=[data_agent],
    tasks=[collect_task, validate_task],
    verbose=1,
    process="workflow"
)

agents.start()

╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: DataCollector                                                                                        │
│  Role: Search Specialist                                                                                        │
│  Tools: internet_search_tool                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ Agent DataCollector is calling function 'internet_search_tool' with arguments: {'query': 'AI job trends in      │
│ 2024'}                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Tool Call ───────────────────────────────────────────────────╮
│ Function 'internet_search_tool' returned: [{"title": "AI in 2024: Five trends workers need to know - BBC",      │
│ "url": "https://www.bbc.com/worklife/article/20240104-ai-in-2024-five-trends-workers-need-to-know", "snippet":  │
│ "A 2023 survey conducted by Jobs for the Future's Center for Artificial Intelligence & the Future of Work (JFF) │
│ showed the majority of respondents believe they will need new skills to compete in an ..."}, {"title": "AI      │
│ Replacing Jobs Statistics: Key Insights and Trends for 2024", "url":                                            │
│ "https://www.allaboutai.com/resources/ai-replacing-jobs-statistics/", "snippet": "AI Could Displace 300 Million │
│ Jobs Globally by 2030. Artificial Intelligence has the potential to displace up to 300 million jobs worldwide   │
│ by 2030, raising concerns about employment and the need for workforce adaptation. ... AllAbout.com Trends and   │
│ Predictions in 2024 and 2030. As we look toward 2024 and 2030, I see AI shaping our careers in ..."}, {"title": │
│ "3 AI Trends That Will Impact Your Job In 2024 - Forbes", "url":                                                │
│ "https://www.forbes.com/sites/rachelwells/2024/09/11/ai-trends-that-will-impact-your-job/", "snippet": "AI is   │
│ creating new job categories and job titles. getty 2. AI Actually Does Boost The Job Market. The second insight  │
│ which is quite eye-opening, reveals that many workers and employers express an ..."}, {"title": "Top 10 AI      │
│ Trends of 2024", "url": "https://aimagazine.com/articles/top-10-ai-trends-of-2024", "snippet": "In light of     │
│ this, AI Magazine presents a comprehensive list of the top 10 AI trends of 2024 to understand the AI            │
│ developments of this year and gain a glimpse of what could be in store for 2025. 10. AI governance and ethics   │
│ ... This dramatic shift in the job market has led to a significant skills shortage, with the International Data │
│ Corporation ..."}, {"title": "131 AI Statistics and Trends for (2024) - National University", "url":            │
│ "https://www.nu.edu/blog/ai-statistics-trends/", "snippet": "In this comprehensive exploration, we delve into   │
│ 131 AI Statistics and Trends (2024), offering a panoramic view of AI's integration into businesses, its         │
│ demographic reach, the burgeoning AI job market, educational initiatives, business owner perspectives, consumer │
│ usage, trust issues, and regulatory discussions."}, {"title": "60+ AI Statistics in Workplace: 2024 Trends and  │
│ Predictions - All About AI", "url": "https://www.allaboutai.com/resources/ai-statistics/workplace/", "snippet": │
│ "(AI statistics 2024) From saving money and boosting productivity to making smarter decisions and redefining    │
│ job roles, AI is making waves across different industries. Explore our comprehensive breakdown of the most      │
│ impactful AI statistics in 2024 and discover how these trends are revolutionizing workplaces worldwide."},      │
│ {"title": "What 2024 Taught Us About AI: Innovations and 2025 Trends", "url":                                   │
│ "https://www.allaboutai.com/resources/what-2024-taught-us-about-ai/", "snippet": "According to AI replacing     │
│ jobs statistics, reports indicated that 40% of jobs globally were influenced by AI in 2024, with sectors like   │
│ finance seeing significant automation. Yet, AI also created roles focused on managing, training, and            │
│ integrating these systems. ... Top 10 AI Trends in 2024. AI evolved rapidly in 2024, with Top 10 AI trends      │
│ ..."}, {"title": "Future of Work Report 2024: AI Will Take Jobs, Make Jobs, and ... - Indeed", "url":           │
│ "https://www.indeed.com/career-advice/news/future-of-work-report-2024", "snippet": "According to Indeed's 2024  │
│ Global Future of Work Report, over 7 in 10 U.S. job se

Output()

Response generated in 13.4s

...

Output()

Response generated in 61.4s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You need to do the following task: Validate the collected data. Check if: 1. At least 5 results are returned.   │
│ 2. Each result contains a title and a URL. Return validation_result as 'valid' or 'invalid' only no other text. │
│ Input data from previous tasks: collect_data: [{"title": "AI in 2024: Five trends workers need to know - BBC",  │
│ "url": "https://www.bbc.com/worklife/article/20240104-ai-in-2024-five-trends-workers-need-to-know", "snippet":  │
│ "A 2023 survey conducted by Jobs for the Future's Center for Artificial Intelligence & the Future of Work (JFF) │
│ showed the majority of respondents believe they will need new skills to compete in an ..."}, {"title": "AI      │
│ Replacing Jobs Statistics: Key Insights and Trends for 2024", "url":                                            │
│ "https://www.allaboutai.com/resources/ai-replacing-jobs-statistics/", "snippet": "AI Could Displace 300 Million │
│ Jobs Globally by 2030. Artificial Intelligence has the potential to displace up to 300 million jobs worldwide   │
│ by 2030, raising concerns about employment and the need for workforce adaptation. ... AllAbout.com Trends and   │
│ Predictions in 2024 and 2030. As we look toward 2024 and 2030, I see AI shaping our careers in ..."}, {"title": │
│ "3 AI Trends That Will Impact Your Job In 2024 - Forbes", "url":                                                │
│ "https://www.forbes.com/sites/rachelwells/2024/09/11/ai-trends-that-will-impact-your-job/", "snippet": "AI is   │
│ creating new job categories and job titles. getty 2. AI Actually Does Boost The Job Market. The second insight  │
│ which is quite eye-opening, reveals that many workers and employers express an ..."}, {"title": "Top 10 AI      │
│ Trends of 2024", "url": "https://aimagazine.com/articles/top-10-ai-trends-of-2024", "snippet": "In light of     │
│ this, AI Magazine presents a comprehensive list of the top 10 AI trends of 2024 to understand the AI            │
│ developments of this year and gain a glimpse of what could be in store for 2025. 10. AI governance and ethics   │
│ ... This dramatic shift in the job market has led to a significant skills shortage, with the International Data │
│ Corporation ..."}, {"title": "131 AI Statistics and Trends for (2024) - National University", "url":            │
│ "https://www.nu.edu/blog/ai-statistics-trends/", "snippet": "In this comprehensive exploration, we delve into   │
│ 131 AI Statistics and Trends (2024), offering a panoramic view of AI's integration into businesses, its         │
│ demographic reach, the burgeoning AI job market, educational initiatives, business owner perspectives, consumer │
│ usage, trust issues, and regulatory discussions."}, {"title": "60+ AI Statistics in Workplace: 2024 Trends and  │
│ Predictions - All About AI", "url": "https://www.allaboutai.com/resources/ai-statistics/workplace/", "snippet": │
│ "(AI statistics 2024) From saving money and boosting productivity to making smarter decisions and redefining    │
│ job roles, AI is making waves across different industries. Explore our comprehensive breakdown of the most      │
│ impactful AI statistics in 2024 and discover how these trends are revolutionizing workplaces worldwide."},      │
│ {"title": "What 2024 Taught Us About AI: Innovations and 2025 Trends", "url":                                   │
│ "https://www.allaboutai.com/resources/what-2024-taught-us-about-ai/", "snippet": "According to AI replacing     │
│ jobs statistics, reports indicated that 40% of jobs globally were influenced by AI in 2024, with sectors like   │
│ finance seeing significant automation. Yet, AI also created roles focused on managing, training, and            │
│ integrating these systems. ... Top 10 AI Trends in 2024. AI evolved rapidly in 2024, with Top 10 AI trends      │
│ ..."}, {"title": "Future of Work Report 2024: AI Will 

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ valid                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'task_status': {0: 'not started', 1: 'not started'},
 'task_results': {0: TaskOutput(description="Perform an internet search using the query: 'AI job trends in 2024'. Return results as a list of title, URL, and snippet.", summary='Perform an', raw='[{"title": "AI in 2024: Five trends workers need to know - BBC", "url": "https://www.bbc.com/worklife/article/20240104-ai-in-2024-five-trends-workers-need-to-know", "snippet": "A 2023 survey conducted by Jobs for the Future\'s Center for Artificial Intelligence & the Future of Work (JFF) showed the majority of respondents believe they will need new skills to compete in an ..."}, {"title": "AI Replacing Jobs Statistics: Key Insights and Trends for 2024", "url": "https://www.allaboutai.com/resources/ai-replacing-jobs-statistics/", "snippet": "AI Could Displace 300 Million Jobs Globally by 2030. Artificial Intelligence has the potential to displace up to 300 million jobs worldwide by 2030, raising concerns about employment and the need for wor